In [1]:
import pandas as pd
import numpy as np

# Introduction to Data Science for Public Policy
## Class 7b: More advanced Pandas - merging and grouping
## Thomas Monk

**Merging** Let's say we have two datasets - note the different sizes of data too.

In [3]:
import pandas as pd
state_years = pd.DataFrame({
'state': ['Ohio', 'Ohio', 'Ohio',
'Nevada', 'Nevada'],
'year': [2000, 2001, 2002, 2001, 2002],
'pop': [1.5, 1.7, 3.6, 2.4, 2.9]
})
display(state_years)

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


In [4]:
states = pd.DataFrame({
'state':['Ohio', 'Utah'],
'abbrev': ['OH', 'UT'],
'area': [4.4, 11.0]
})
display(states)

,state,abbrev,area
0,Ohio,OH,4.4
1,Utah,UT,11.0


What if I wanted to know the population density of each state in each year? Thoughts?

**Merge function** The merge function allows us to combine two datasets that share a column:

In [6]:
df_new = state_years.merge(states, on='state')
df_new

,state,year,pop,abbrev,area
0,Ohio,2000,1.5,OH,4.4
1,Ohio,2001,1.7,OH,4.4
2,Ohio,2002,3.6,OH,4.4


- The state column is called the merge key.
- The state_years table is called the left and states is called
the right.
- The rows corresponding to Nevada and Utah don’t appear in
the result because the merge function performs an inner merge
by default.

**Merge `how` Venn diagrams**
![](https://miro.medium.com/max/1200/1*9eH1_7VbTZPZd9jBiGIyNA.png)

**inner merge**
The inner merge only includes rows that are
present on both sides of the merge:

In [14]:
state_years.merge(states, on='state', how='inner')

,state,year,pop,abbrev,area
0,Ohio,2000,1.5,OH,4.4
1,Ohio,2001,1.7,OH,4.4
2,Ohio,2002,3.6,OH,4.4


**left merge**
The left merge includes all rows from the left table, producing nulls
when a key is missing on the right:

In [16]:
state_years.merge(states, on='state', how='left')

,state,year,pop,abbrev,area
0,Ohio,2000,1.5,OH,4.4
1,Ohio,2001,1.7,OH,4.4
2,Ohio,2002,3.6,OH,4.4
3,Nevada,2001,2.4,NaN,NaN
4,Nevada,2002,2.9,NaN,NaN


**right merge**
A right merge includes all rows from the right table, producing nulls
when a key is missing on the left:

In [18]:
state_years.merge(states, on='state', how='right')

,state,year,pop,abbrev,area
0,Ohio,2000.0,1.5,OH,4.4
1,Ohio,2001.0,1.7,OH,4.4
2,Ohio,2002.0,3.6,OH,4.4
3,Utah,NaN,NaN,UT,11.0


**outer merge** An outer merge includes all rows, producing nulls when a ket from
the left is missing on the right and vice versa:

In [20]:
state_years.merge(states, on='state', how='outer')

,state,year,pop,abbrev,area
0,Ohio,2000.0,1.5,OH,4.4
1,Ohio,2001.0,1.7,OH,4.4
2,Ohio,2002.0,3.6,OH,4.4
3,Nevada,2001.0,2.4,NaN,NaN
4,Nevada,2002.0,2.9,NaN,NaN
5,Utah,NaN,NaN,UT,11.0


**A merge in practice**
Let's say that we wanted to have a column in our salaries database which told us the size of each department. For example:
![](06.jpg)

Any thoughts on how we could do this? Think merges.

Let's initially look at the Department sizes.

In [27]:
salaries = pd.read_csv('../Class 6/salaries.csv')
dept_sizes = salaries.Department.value_counts()
display(dept_sizes.head(5))

POLICE           13414
FIRE              4641
STREETS & SAN     2198
OEMC              2102
WATER MGMNT       1879
Name: Department, dtype: int64

What data type is this?

This is a series, so we can't merge with it yet. Let's transform it into a dataframe. We do this simply by adding ('resetting') the index - the series becomes a Data Frame.

In [32]:
depts = dept_sizes.reset_index()
display(depts.head(5)) # What next?

,index,Department
0,POLICE,13414
1,FIRE,4641
2,STREETS & SAN,2198
3,OEMC,2102
4,WATER MGMNT,1879


The column names are confusing - let's rename them.

In [34]:
depts.columns = ['Department', 'Department Size']
display(depts.head(5))

,Department,Department Size
0,POLICE,13414
1,FIRE,4641
2,STREETS & SAN,2198
3,OEMC,2102
4,WATER MGMNT,1879


Next?

In [36]:
salaries.merge(depts,on=['Department'])

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate,Department Size
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,$101442.00,NaN,13414
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,$94122.00,NaN,13414
2,"ABBATE, TERRY M",POLICE OFFICER,POLICE,F,Salary,NaN,$93354.00,NaN,13414
3,"ABDALLAH, ZAID",POLICE OFFICER,POLICE,F,Salary,NaN,$84054.00,NaN,13414
4,"ABDELHADI, ABDALMAHD",POLICE OFFICER,POLICE,F,Salary,NaN,$87006.00,NaN,13414
...,...,...,...,...,...,...,...,...,...
33178,"TADEO, ALITA F",ADMINISTRATIVE ASST II,ADMIN HEARNG,F,Salary,NaN,$63876.00,NaN,39
33179,"TAKATA, NANCY M",EXEC ADMINISTRATIVE ASST II,ADMIN HEARNG,F,Salary,NaN,$70620.00,NaN,39
33180,"WEINERT, DARCI N",SENIOR ADMINISTRATIVE LAW OFFICER,ADMIN HEARNG,F,Salary,NaN,$96636.00,NaN,39
33181,"WOODRIDGE, RACHENETTE",ADMINISTRATIVE ASST II,ADMIN HEARNG,F,Salary,NaN,$63876.00,NaN,39


Now, let's look back at the Question 4 of the Problem Set of Class 6, and work on these again. We now know everything we need to know.

**Homework** PS7. Applies everything from this lecture, including merging, on the Chicago crime database.